In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
from cv2 import cv2
import random
#from keras import bakend as k
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.losses import binary_crossentropy
from keras.objectives import mean_squared_error,mean_absolute_percentage_error
import keras
from keras.callbacks import *
import matplotlib.pyplot as plt
from keras.applications import MobileNetV2
from keras.optimizers import Adam, RMSprop, SGD
from keras.models import load_model
from sklearn.model_selection import KFold,train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout, Conv2D,Conv2DTranspose, BatchNormalization, Activation,AveragePooling2D,MaxPooling2D,Flatten,GlobalAveragePooling2D, Input, Concatenate, MaxPool2D, Add, UpSampling2D, LeakyReLU,ZeroPadding2D
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler

In [ ]:
df_train=pd.read_csv("/kaggle/input/landmark-recognition-2020/train.csv")
df_train.head(5)

In [ ]:
df_train["img"]=df_train.id.str[0]+"/"+df_train.id.str[1]+"/"+df_train.id.str[2]+"/"+df_train.id+".jpg"
df_train.head(5)

In [ ]:
df_train["label"]=df_train["landmark_id"].values.astype("str")
df_train.head(5)

In [ ]:
df_train.drop(["landmark_id"],axis=1,inplace=True)
df_train.head(5)

In [ ]:
df_train.drop(["id"],axis=1,inplace=True)
df_train.head(5)

In [ ]:
df_train.sample(frac=1)

In [ ]:
df_train.drop(df_train.tail(1000).img,inplace=True,axis=0)

In [ ]:
df_train


In [ ]:
df_train.drop(df_train.tail(500000).index,inplace=True)

In [ ]:
df_train

In [ ]:
df_train.drop(df_train.tail(69470).index,inplace=True)
df_train

In [ ]:
df_train.img[df_train.index[1]]

In [ ]:
df_train.resize(1000000,2)

In [ ]:
datagen=ImageDataGenerator(validation_split=0.25,samplewise_center=True, samplewise_std_normalization=True)
train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/landmark-recognition-2020/train/",
x_col="img",
y_col="label",
subset="training",
batch_size=512,
shuffle=True,
class_mode="categorical",
color_mode="rgb",
target_size=(224,224)
)
valid_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/landmark-recognition-2020/train/",
x_col="img",
y_col="label",
subset="validation",
batch_size=512,
shuffle=True,
color_mode="rgb",
class_mode="categorical",
target_size=(224,224)
)



In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.callbacks import ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

from keras import backend as K

In [ ]:
def relu6(x):
    return K.relu(x,max_value=6.0)

In [ ]:
def bottleneck(inputs, filters, kernel, t, s, r=False):
    tchannel = K.int_shape(inputs)[1] * t
    x=Conv2D(tchannel,(1,1),padding="same",strides=(1,1))(inputs)
    x=BatchNormalization()(x)
    x=Activation(relu6)(x)
    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    if r:
        x = Add()([x, inputs])

    return x

In [ ]:
def inverted_residual_block(inputs, filters, kernel, t, strides, n):    
    x = bottleneck(inputs, filters, kernel, t, strides)
    for i in range(1, n):
        x = bottleneck(x, filters, kernel, t, 1, True)
    return x

In [ ]:
def mobilenet(inputshape,k):
    inputs=Input(inputshape)
    first_filter=32
    x=Conv2D(32,(3,3),padding="same",strides=(2,2))(inputs)
    x=BatchNormalization()(x)
    x=Activation(relu6)(x)
    x = inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = inverted_residual_block(x, 320, (3, 3), t=6,  strides=1, n=1)
    last_filters=1280
    x=Conv2D(1280,(1,1),padding="same",strides=(1,1))(x)
    x=BatchNormalization()(x)
    x=Activation(relu6)(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)
    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)
    model = Model(inputs, output)
    return model

In [ ]:
model_l = Sequential()
model_l.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(224,224,3)))
model_l.add(BatchNormalization())
model_l.add(Conv2D(32,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))
model_l.add(Conv2D(64,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(64,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))
model_l.add(Flatten())
model_l.add(Dense(128, activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))
model_l.add(Dense(25753, activation='softmax'))


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1026))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(81313, activation='softmax'))
model.summary()

In [ ]:
#model_x=load_model("/kaggle/input/common-keras-pretrained-models/MobileNetV2.h5")
#model=mobilenet((224,224,3),81313)
learning_rate=0.00001
lrr=ReduceLROnPlateau(monitor="accuracy",patience=2,verbose=1,factor=0.5,min_lr=0.0000001)
model_l.compile(optimizer=Adam(lr=learning_rate),loss="categorical_crossentropy",metrics=["accuracy"])
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
hist=model_l.fit(train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
               batch_size=512,
                    epochs=1,
              verbose=1,callbacks=[lrr]
)
model.save_weights("save_weights.h5")